In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import time

import PIL
import matplotlib.pyplot as plt

In [ ]:
classifier_model = "https://hub.tensorflow.google.cn/google/tf2-preview/mobilenet_v2/classification/4"

In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape = IMAGE_SHAPE + (3, ))
])

In [ ]:
classifier.summary()

In [ ]:
grace_hopper = tf.keras.utils.get_file('image.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = PIL.Image.open(grace_hopper).resize(IMAGE_SHAPE)

In [ ]:
grace_hopper

In [ ]:
grace_hopper = np.array(grace_hopper) / 255.0

In [ ]:
grace_hopper.shape

In [ ]:
result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

In [ ]:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
imagenet_labels

In [ ]:
plt.imshow(grace_hopper)
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title('Prediction: ' + predicted_class_name)

In [ ]:
data_root = tf.keras.utils.get_file('flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', untar=True)

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(str(data_root), validation_split=0.2, subset='training', seed=42, image_size=(img_height, img_width), batch_size=batch_size)

In [ ]:
class_names = np.array(train_ds.class_names)
class_names

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
train_ds

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(AUTOTUNE)

In [ ]:
for image_batch, label_batch in train_ds:
    print(image_batch)
    print(image_batch.shape)
    print(label_batch.shape)
    break

In [ ]:
result_batch = classifier.predict(train_ds)

In [ ]:
predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

In [ ]:
plt.figure(figsize=(12, 10))
plt.subplots_adjust(hspace=0.5)

for i in range(30):
    plt.subplot(6, 5, i + 1)
    plt.imshow(image_batch[i])
    plt.title(predicted_class_names[i])
_ = plt.suptitle('Imagenet prediction')

In [ ]:
feature_extrator_model = "https://hub.tensorflow.google.cn/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
feature_extrator_layer = hub.KerasLayer(feature_extrator_model, input_shape = (224, 224, 3), trainable = False)

In [ ]:
feature_batch = feature_extrator_layer(image_batch)
feature_batch.shape

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
    feature_extrator_layer,
    tf.keras.layers.Dense(num_classes),
])

In [ ]:
model.summary()

In [ ]:
predictions = model(image_batch)

In [ ]:
predictions.shape

In [ ]:
model.compile('adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(True), metrics=['acc'])

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_acc = []
    
    def on_train_batch_end(self, batch, logs = None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['acc'])
        self.model.reset_metrics()

In [ ]:
batch_stats_callback = CollectBatchStats()

history = model.fit(train_ds, epochs=2, callbacks=[batch_stats_callback])

In [ ]:
plt.figure()
plt.ylabel('Loss')
plt.xlabel('Thain step')
plt.ylim([0, 2])
plt.plot(batch_stats_callback.batch_losses)
plt.show()

In [ ]:
plt.figure()
plt.ylabel('Acc')
plt.xlabel('Train step')
plt.ylim([0, 1])
plt.plot(batch_stats_callback.batch_acc)
plt.show()

In [ ]:
predicted_batch = model.predict(image_batch)

In [ ]:
predicted_batch

In [ ]:
predicted_id = np.argmax(predicted_batch, axis = -1)
predicted_id

In [ ]:
predicted_label_batch = class_names[predicted_id]
predicted_label_batch

In [ ]:
plt.figure(figsize=(12, 10))
plt.subplots_adjust(hspace=0.5)
for i in range(30):
    plt.subplot(6, 5, i + 1)
    plt.imshow(image_batch[i])
    plt.title(predicted_label_batch[i])

_ = plt.suptitle('Model prediction')

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(str(data_root), validation_split=0.2, subset='validation', seed=42, image_size=(img_height, img_width), batch_size=batch_size)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.cache().prefetch(AUTOTUNE)

In [ ]:
loss, acc = model.evaluate(val_ds)

In [ ]:
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path)

export_path

In [ ]:
reloaded = tf.keras.models.load_model(export_path)

loss, acc = reloaded.evaluate(val_ds)